# Generate the agent
This notebook latter we will pass it into the `base agents.py`

In [42]:
#Imports
import os
import json
#Typing
from typing import Dict, List, Any, Optional
from IPython.display import Image, display

#LangGraph/LangChain
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI  # or any other LLM provider
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
#Load env files
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [12]:
llm = ChatOpenAI(model="gpt-4.1-mini")


In [13]:
class MainState(Dict):
    messages: Dict[str, Any]
    current_fable : str
    processing_request : Dict[str, Any]
    tool_output : Dict[str, Any]
    final_story : str

In [45]:
# Define the tool(fable aesop)

# Cell 2: Add debug prints to aesop_fable_tool
def aesop_fable_tool(state: MainState) -> Dict[str, Any]:
    """
    Tool for processing Aesop's fables using LLM
    """
    print("\n=== Aesop Tool Debug ===")
    current_fable = state.get("current_fable", "")
    request = state.get("processing_request", {})
    action = request.get("action", "analyze")
    
    print(f"Current fable: {current_fable[:50]}...")
    print(f"Processing request: {request}")
    print(f"Action: {action}")
    
    if action == "analyze":
        # Use LLM to analyze the fable
        system_prompt = """You are an expert in Aesop's fables. Analyze the given fable and provide:
        1. The main moral/lesson
        2. List of characters (animals/humans)
        3. A brief analysis of how the moral is conveyed
        4. The story structure (beginning, conflict, resolution)
        
        Respond in JSON format."""
        
        response = llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Analyze this fable:\n\n{current_fable}")
        ])
        
        print(f"LLM Response: {response.content}")
        
        # Parse the response (assuming JSON format)
        try:
            analysis = json.loads(response.content)
        except:
            # Fallback if JSON parsing fails
            analysis = {
                "moral": response.content,
                "characters": [],
                "analysis": "",
                "structure": ""
            }
            print("JSON parsing failed, using fallback")
            
        output = {
            "original_fable": current_fable,
            "analysis": analysis,
            "word_count": len(current_fable.split())
        }
    else:
        output = {"message": f"Action {action} not implemented in debug"}
    
    print(f"Tool output: {output}")
    return {"tool_output": output}

In [44]:
# Cell 1: Add debug prints to main_agent
def main_agent(state: MainState) -> Dict[str, Any]:
    """
    Main orchestrator agent that uses LLM to decide which tool to use
    """
    print("\n=== Main Agent Debug ===")
    current_fable = state.get("current_fable", "")
    messages = state.get("messages", [])
    
    print(f"Current fable: {current_fable[:50]}...")
    print(f"Messages: {messages}")
    
    # Use LLM to analyze the user's intent and the content
    system_prompt = """You are an AI orchestrator for a story processing system. 
    Analyze the user's request and the provided text to determine:
    1. Is this an Aesop's fable? (yes/no)
    2. What action should be taken? (analyze/retell/expand/modernize/create_new)
    3. Any specific requirements? (style, moral, characters)
    
    Respond in JSON format with keys: is_aesop, action, requirements"""
    
    user_message = messages[-1].get("content", "") if messages else ""
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"User request: {user_message}\n\nText: {current_fable}")
    ])
    
    print(f"LLM Response: {response.content}")
    
    try:
        decision = json.loads(response.content)
    except:
        # Fallback decision
        decision = {
            "is_aesop": True,
            "action": "analyze",
            "requirements": {}
        }
        print("JSON parsing failed, using fallback")
    
    print(f"Decision: {decision}")
    
    if decision.get("is_aesop", True):
        processing_request = {
            "action": decision.get("action", "analyze"),
            **decision.get("requirements", {})
        }
        
        result = {
            "processing_request": processing_request,
            "tool_to_call": "aesop_fable_tool"
        }
        print(f"Returning: {result}")
        return result
    else:
        return {
            "processing_request": {"action": "analyze"},
            "tool_to_call": "generic_story_tool"
        }
        

In [46]:
# Cell 3: Add debug to tool_router
def tool_router(state: MainState) -> Dict[str, Any]:
    """
    Routes to the appropriate tool based on main agent's decision
    """
    print("\n=== Tool Router Debug ===")
    print(f"State keys: {state.keys()}")
    print(f"Tool to call: {state.get('tool_to_call', 'Not set')}")
    print(f"Processing request: {state.get('processing_request', {})}")
    
    tool_name = state.get("tool_to_call", "aesop_fable_tool")
    
    if tool_name == "aesop_fable_tool":
        return aesop_fable_tool(state)
    else:
        return {"tool_output": {"error": "Tool not found"}}

In [47]:
def generate_final_output(state: MainState) -> Dict[str, Any]:
    """
    Formats the final output for the user using LLM for polish
    """
    tool_output = state.get("tool_output", {})
    action = state.get("processing_request", {}).get("action", "analyze")
    
    if "error" in tool_output:
        final_story = f"Error: {tool_output['error']}"
    else:
        # Use LLM to format the output nicely
        system_prompt = f"""You are a helpful assistant. Format the following {action} output 
        in a clear, readable way for the user. Make it engaging and well-structured."""
        
        response = llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Format this output:\n{json.dumps(tool_output, indent=2)}")
        ])
        
        final_story = response.content
    
    return {"final_story": final_story}

In [48]:

# Build graph
builder = StateGraph(MainState)

# Add nodes
builder.add_node("main_agent", main_agent)
builder.add_node("tool_router", tool_router)
builder.add_node("generate_output", generate_final_output)

# Logic
builder.add_edge(START, "main_agent")
builder.add_edge("main_agent", "tool_router")
builder.add_edge("tool_router", "generate_output")
builder.add_edge("generate_output", END)

# Compile
graph = builder.compile()


# Cell 8: Build and visualize

# View
print(graph.get_graph().draw_ascii())


   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
  +------------+     
  | main_agent |     
  +------------+     
          *          
          *          
          *          
  +-------------+    
  | tool_router |    
  +-------------+    
          *          
          *          
          *          
+-----------------+  
| generate_output |  
+-----------------+  
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      


In [49]:
# Cell 8: Test the system
app = build_story_graph()


In [50]:

# Cell 1: Test Basic Analysis
test_fable = """
The Fox and the Grapes

A hungry Fox saw some fine bunches of Grapes hanging from a vine that was trained along a high trellis, and did his best to reach them by jumping as high as he could into the air. But it was all in vain, for they were just out of reach: so he gave up trying, and walked away with an air of dignity and unconcern, remarking, "I thought those Grapes were ripe, but I see now they are quite sour."
"""

# Test 1: Basic analysis
analysis_state = {
    "messages": [{"role": "user", "content": "Analyze this fable and tell me about the moral"}],
    "current_fable": test_fable,
    "processing_request": {},
    "tool_output": {},
    "final_story": ""
}

print("Test 1: Analysis")
print("-" * 50)
result = graph.invoke(analysis_state)
print(result["final_story"])
print("\n")

Test 1: Analysis
--------------------------------------------------

=== Main Agent Debug ===
Current fable: 
The Fox and the Grapes

A hungry Fox saw some fin...
Messages: [{'role': 'user', 'content': 'Analyze this fable and tell me about the moral'}]
LLM Response: {
  "is_aesop": "yes",
  "action": "analyze",
  "requirements": {
    "explanation": "Explain the moral of the fable."
  }
}
Decision: {'is_aesop': 'yes', 'action': 'analyze', 'requirements': {'explanation': 'Explain the moral of the fable.'}}
Returning: {'processing_request': {'action': 'analyze', 'explanation': 'Explain the moral of the fable.'}, 'tool_to_call': 'aesop_fable_tool'}

=== Tool Router Debug ===
State keys: dict_keys(['messages', 'current_fable', 'processing_request', 'tool_output', 'final_story'])
Tool to call: Not set
Processing request: {'action': 'analyze', 'explanation': 'Explain the moral of the fable.'}

=== Aesop Tool Debug ===
Current fable: 
The Fox and the Grapes

A hungry Fox saw some fin...
Proce